In [1]:
import numpy as np
import sys
sys.path.append("../")
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.theodolite_function as theodo_f

# Important: set path to deployment data
path = "../data/20220525-1/"

In [2]:
# Function to read sensors extrinsic calibration file and see the inter-prism and inter-GNSS distance (if GNSS was used during the deployment)
theodo_u = importlib.reload(theodo_u)

extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(path+"sensors_extrinsic_calibration/calibration_results.csv")
print("Inter-prism distance measured [m]: ", extrinsic_calibration_results[0:3])
if(len(extrinsic_calibration_results)>3):
    print("Inter-GNSS distance measured [m]: ", extrinsic_calibration_results[3:6])

Inter-prism distance measured [m]:  [0.9069738865416104, 0.734951292757258, 1.027486624635416]
Inter-GNSS distance measured [m]:  [0.7922879928784302, 0.8393905729950949, 0.5168505712794828]


In [7]:
# Function used to show the resection accuracy
# Read markers file of grand axe and show them

%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)

path_file_GCP = "total_stations/GCP.txt"
file_name = path+path_file_GCP

trimble_1_gcp, trimble_2_gcp, trimble_3_gcp, T_1_grand, T_2_grand, T_3_grand = theodo_u.read_marker_file(file_name, 1, 1)

tp1 = T_1_grand@trimble_1_gcp
tp2 = T_2_grand@trimble_2_gcp
tp3 = T_3_grand@trimble_3_gcp

print(T_2_grand)


# plot_3d = 0   # Variable to plot in 3d, 0 = 2D, 1 = 3D
# save_pdf = 0   # Variable to save file in pdf
# file_pdf = path + "figures/GCP_calibration_result.pdf"   # file name to save plot in pdf
# theodo_p.plot_trajectories_prism(3, tp1, tp2, tp3, T_1_grand, T_2_grand, T_3_grand, plot_3d, save_pdf,file_pdf,1)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 9.23087351e-01  3.84590357e-01  4.75548934e-06 -1.23523655e+00]
 [-3.84590355e-01  9.23087347e-01  9.43932272e-05 -7.45555639e-01]
 [ 3.19129929e-05 -8.89621093e-05  9.99999996e-01  2.81086312e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [61]:
from scipy.spatial.transform import Rotation as R_scipy

num = 100
r = R_scipy.from_matrix(T_2_grand[0:3, 0:3])
angle = r.as_euler('xyz', degrees=False)
t = T_2_grand[0:3,3]
T_sample_2 = []
for i in range(0,num+4):
    if(i<=num):
        angle_corrected = np.array([angle[0]*i/num,angle[1]*i/num,angle[2]*i/num])
        r = R_scipy.from_euler('xyz', angle_corrected.flatten(), degrees=False)
        R_corrected = r.as_matrix()
        T_corrected = np.identity(4)
        T_corrected[0:3,0:3]= R_corrected
        T_change = np.array([t[0]*i/num,t[1]*i/num,t[2]*i/num,1]).T
        T_corrected[:,3] = T_change
        T_sample_2.append(T_corrected)
    else:
        T_sample_2.append(T_2_grand)

r = R_scipy.from_matrix(T_3_grand[0:3, 0:3])
angle = r.as_euler('xyz', degrees=False)
t = T_3_grand[0:3,3]
T_sample_3 = []
for i in range(0,num+4):
    if(i<=num):
        angle_corrected = np.array([angle[0]*i/num,angle[1]*i/num,angle[2]*i/num])
        r = R_scipy.from_euler('xyz', angle_corrected.flatten(), degrees=False)
        R_corrected = r.as_matrix()
        T_corrected = np.identity(4)
        T_corrected[0:3,0:3]= R_corrected
        T_change = np.array([t[0]*i/num,t[1]*i/num,t[2]*i/num,1]).T
        T_corrected[:,3] = T_change
        T_sample_3.append(T_corrected)
    else:
        T_sample_3.append(T_3_grand)

In [69]:
# Important: change path to rosbag
file = "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220525/20220525_1_inter_prism.bag"

_, _, _, ttrimble_1, ttrimble_2, ttrimble_3, _, _, _, _, _, _, _, _, _ = theodo_u.read_rosbag_theodolite_without_tf_raw_data(file)
print(ttrimble_1)

Number of data for theodolites: [4214 3791 3347]
Bad measures: 924
[[1.03194864 1.03191921 1.03198701 ... 0.3268109  0.32686117 0.32690572]
 [3.24917387 3.24906902 3.24929646 ... 3.95212166 3.95260485 3.95302868]
 [0.51823153 0.51821553 0.51825151 ... 0.53414078 0.53420568 0.53426371]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [68]:
## Animation GCP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

tp1 = T_1_grand@trimble_1_gcp
tp2 = np.array(T_sample_2[0])@trimble_2_gcp
tp3 = np.array(T_sample_3[0])@trimble_3_gcp
# Theodolite positions in theodolite 1 frame
ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
ax.scatter(T_sample_2[0][0,3], T_sample_2[0][1,3], c='blue', marker='*', s=200, label="RTS 2")
ax.scatter(T_sample_3[0][0,3], T_sample_3[0][1,3], c='green', marker='*', s=200, label="RTS 3")
# Prisms trajectories in theodolite 1 frame
ax.scatter(tp1[0], tp1[1], c='r', marker='.', label="GCP 1")
ax.scatter(tp2[0], tp2[1], c='b', marker='.', label="GCP 2")
ax.scatter(tp3[0], tp3[1], c='g', marker='.', label="GCP 3")

plt.xlabel('x[m]')
plt.ylabel('y[m]')
plt.xlim(-20, 20)
plt.ylim(-15,30)

# sc_1 = ax.scatter([], [], color="green", zorder=4)
# line, = ax.plot([], [], color="crimson", zorder=4)
# sc_2 = ax.scatter([], [], color="gold", zorder=4)

def animate(i):
    if(i<=num+3):
        tp1 = T_1_grand@trimble_1_gcp
        tp2 = np.array(T_sample_2[i])@trimble_2_gcp
        tp3 = np.array(T_sample_3[i])@trimble_3_gcp
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_sample_2[i][0,3], T_sample_2[i][1,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_sample_3[i][0,3], T_sample_3[i][1,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tp1[0], tp1[1], c='r', marker='.', label="GCP 1")
        ax.scatter(tp2[0], tp2[1], c='b', marker='.', label="GCP 2")
        ax.scatter(tp3[0], tp3[1], c='g', marker='.', label="GCP 3")
    else:
        tp1 = T_1_grand@trimble_1_gcp
        tp2 = T_2_grand@trimble_2_gcp
        tp3 = T_3_grand@trimble_3_gcp
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_2_grand[0,3], T_2_grand[1,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_3_grand[0,3], T_3_grand[1,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tp1[0], tp1[1], c='r', marker='.', label="GCP 1")
        ax.scatter(tp2[0], tp2[1], c='b', marker='.', label="GCP 2")
        ax.scatter(tp3[0], tp3[1], c='g', marker='.', label="GCP 3")
    plt.xlabel('x[m]')
    plt.ylabel('y[m]')
    plt.legend(loc='upper left')
    plt.xlim(-20, 20)
    plt.ylim(-15,30)

ani = animation.FuncAnimation(fig=fig, func=animate, frames=num+20, interval=20, blit=False)
ani.save('/home/maxime/test.mp4')
plt.show()


<IPython.core.display.Javascript object>

In [73]:
## Animation Raw data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

tp1 = T_1_grand@trimble_1_gcp
tp2 = np.array(T_sample_2[0])@trimble_2_gcp
tp3 = np.array(T_sample_3[0])@trimble_3_gcp
tt1 = T_1_grand@ttrimble_1
tt2 = np.array(T_sample_2[0])@ttrimble_2
tt3 = np.array(T_sample_3[0])@ttrimble_3
# Theodolite positions in theodolite 1 frame
ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
ax.scatter(T_sample_2[0][0,3], T_sample_2[0][1,3], c='blue', marker='*', s=200, label="RTS 2")
ax.scatter(T_sample_3[0][0,3], T_sample_3[0][1,3], c='green', marker='*', s=200, label="RTS 3")
# Prisms trajectories in theodolite 1 frame
ax.scatter(tp1[0], tp1[1], c='black', marker='.', label="GCP")
ax.scatter(tp2[0], tp2[1], c='black', marker='.')
ax.scatter(tp3[0], tp3[1], c='black', marker='.')
ax.scatter(tt1[0], tt1[1], c='r', marker='.', label="Prism 1")
ax.scatter(tt2[0], tt2[1], c='b', marker='.', label="Prism 2")
ax.scatter(tt3[0], tt3[1], c='g', marker='.', label="Prism 3")

plt.xlabel('x[m]')
plt.ylabel('y[m]')
plt.xlim(-50, 60)
plt.ylim(-15,70)

# sc_1 = ax.scatter([], [], color="green", zorder=4)
# line, = ax.plot([], [], color="crimson", zorder=4)
# sc_2 = ax.scatter([], [], color="gold", zorder=4)

def animate(i):
    if(i<=num+3):
        tp1 = T_1_grand@trimble_1_gcp
        tp2 = np.array(T_sample_2[i])@trimble_2_gcp
        tp3 = np.array(T_sample_3[i])@trimble_3_gcp
        tt1 = T_1_grand@ttrimble_1
        tt2 = np.array(T_sample_2[i])@ttrimble_2
        tt3 = np.array(T_sample_3[i])@ttrimble_3
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_sample_2[i][0,3], T_sample_2[i][1,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_sample_3[i][0,3], T_sample_3[i][1,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tp1[0], tp1[1], c='black', marker='.', label="GCP")
        ax.scatter(tp2[0], tp2[1], c='black', marker='.')
        ax.scatter(tp3[0], tp3[1], c='black', marker='.')
        ax.scatter(tt1[0], tt1[1], c='r', marker='.', label="Prism 1")
        ax.scatter(tt2[0], tt2[1], c='b', marker='.', label="Prism 2")
        ax.scatter(tt3[0], tt3[1], c='g', marker='.', label="Prism 3")
    else:
        tp1 = T_1_grand@trimble_1_gcp
        tp2 = T_2_grand@trimble_2_gcp
        tp3 = T_3_grand@trimble_3_gcp
        tt1 = T_1_grand@ttrimble_1
        tt2 = T_2_grand@ttrimble_2
        tt3 = T_3_grand@ttrimble_3
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[0,3], T_1_grand[1,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_2_grand[0,3], T_2_grand[1,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_3_grand[0,3], T_3_grand[1,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tp1[0], tp1[1], c='black', marker='.', label="GCP")
        ax.scatter(tp2[0], tp2[1], c='black', marker='.')
        ax.scatter(tp3[0], tp3[1], c='black', marker='.')
        ax.scatter(tt1[0], tt1[1], c='r', marker='.', label="Prism 1")
        ax.scatter(tt2[0], tt2[1], c='b', marker='.', label="Prism 2")
        ax.scatter(tt3[0], tt3[1], c='g', marker='.', label="Prism 3")
    plt.xlabel('x[m]')
    plt.ylabel('y[m]')
    plt.legend(loc='upper left')
    plt.xlim(-50, 60)
    plt.ylim(-15, 70)

ani = animation.FuncAnimation(fig=fig, func=animate, frames=num+20, interval=20, blit=False)
ani.save('/home/maxime/test_prism.mp4')
plt.show()

<IPython.core.display.Javascript object>

In [75]:
path_type = "filtered_prediction/"
file_name = "f-2-1-1-1-6-0-L_"

trimble_1 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"1.csv"))
trimble_2 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"2.csv"))
trimble_3 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"3.csv"))

In [100]:
## Animation Raw data without CGP
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

index = np.round(np.linspace(0,len(trimble_1)-1,100))
axis_1 = 0
axis_2 = 2

tt1 = T_1_grand@trimble_1.T[1:]
tt2 = np.array(T_sample_2[0])@trimble_2.T[1:]
tt3 = np.array(T_sample_3[0])@trimble_3.T[1:]
# Theodolite positions in theodolite 1 frame
ax.scatter(T_1_grand[axis_1,3], T_1_grand[axis_2,3], c='red', marker='*', s=200, label="RTS 1")
ax.scatter(T_sample_2[0][axis_1,3], T_sample_2[0][axis_2,3], c='blue', marker='*', s=200, label="RTS 2")
ax.scatter(T_sample_3[0][axis_1,3], T_sample_3[0][axis_2,3], c='green', marker='*', s=200, label="RTS 3")
# Prisms trajectories in theodolite 1 frame
ax.scatter(tt1[axis_1], tt1[1], c='r', marker='.', label="Prism 1")
ax.scatter(tt2[axis_1], tt2[1], c='b', marker='.', label="Prism 2")
ax.scatter(tt3[axis_1], tt3[1], c='g', marker='.', label="Prism 3")
for i in index:
    ax.plot([tt1[axis_1][int(i)], tt2[axis_1][int(i)]], [tt1[axis_2][int(i)], tt2[axis_2][int(i)]], 'black', linewidth=1)
    ax.plot([tt1[axis_1][int(i)], tt3[axis_1][int(i)]], [tt1[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)
    ax.plot([tt2[axis_1][int(i)], tt3[axis_1][int(i)]], [tt2[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)

plt.xlabel('x[m]')
plt.ylabel('y[m]')
plt.xlim(-50, 60)
plt.ylim(-15,70)

# sc_1 = ax.scatter([], [], color="green", zorder=4)
# line, = ax.plot([], [], color="crimson", zorder=4)
# sc_2 = ax.scatter([], [], color="gold", zorder=4)

def animate(i):
    if(i<=num+3):
        tt1 = T_1_grand@trimble_1.T[1:]
        tt2 = np.array(T_sample_2[i])@trimble_2.T[1:]
        tt3 = np.array(T_sample_3[i])@trimble_3.T[1:]
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[axis_1,3], T_1_grand[axis_2,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_sample_2[i][axis_1,3], T_sample_2[i][axis_2,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_sample_3[i][axis_1,3], T_sample_3[i][axis_2,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tt1[axis_1], tt1[axis_2], c='r', marker='.', label="Prism 1")
        ax.scatter(tt2[axis_1], tt2[axis_2], c='b', marker='.', label="Prism 2")
        ax.scatter(tt3[axis_1], tt3[axis_2], c='g', marker='.', label="Prism 3")
        for i in index:
            ax.plot([tt1[axis_1][int(i)], tt2[axis_1][int(i)]], [tt1[axis_2][int(i)], tt2[axis_2][int(i)]], 'black', linewidth=1)
            ax.plot([tt1[axis_1][int(i)], tt3[axis_1][int(i)]], [tt1[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)
            ax.plot([tt2[axis_1][int(i)], tt3[axis_1][int(i)]], [tt2[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)
    else:
        tt1 = T_1_grand@trimble_1.T[1:]
        tt2 = T_2_grand@trimble_2.T[1:]
        tt3 = T_3_grand@trimble_3.T[1:]
        plt.cla()
        # Theodolite positions in theodolite 1 frame
        ax.scatter(T_1_grand[axis_1,3], T_1_grand[axis_2,3], c='red', marker='*', s=200, label="RTS 1")
        ax.scatter(T_2_grand[axis_1,3], T_2_grand[axis_2,3], c='blue', marker='*', s=200, label="RTS 2")
        ax.scatter(T_3_grand[axis_1,3], T_3_grand[axis_2,3], c='green', marker='*', s=200, label="RTS 3")
        # Prisms trajectories in theodolite 1 frame
        ax.scatter(tt1[axis_1], tt1[axis_2], c='r', marker='.', label="Prism 1")
        ax.scatter(tt2[axis_1], tt2[axis_2], c='b', marker='.', label="Prism 2")
        ax.scatter(tt3[axis_1], tt3[axis_2], c='g', marker='.', label="Prism 3")
        for i in index:
            ax.plot([tt1[axis_1][int(i)], tt2[axis_1][int(i)]], [tt1[axis_2][int(i)], tt2[axis_2][int(i)]], 'black', linewidth=1)
            ax.plot([tt1[axis_1][int(i)], tt3[axis_1][int(i)]], [tt1[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)
            ax.plot([tt2[axis_1][int(i)], tt3[axis_1][int(i)]], [tt2[axis_2][int(i)], tt3[axis_2][int(i)]], 'black', linewidth=1)
    plt.xlabel('x[m]')
    plt.ylabel('y[m]')
    plt.legend(loc='upper left')
    plt.xlim(-50, 60)
    plt.ylim(-15, 70)

ani = animation.FuncAnimation(fig=fig, func=animate, frames=num+20, interval=20, blit=False)
ani.save('/home/maxime/test_prism_without_GCP2.mp4')
plt.show()

<IPython.core.display.Javascript object>